In [5]:
# Porting command line interface over to Jupyter notebook
master_dir = '' #'stuff_for_jihoon_2021-06-04/'
output_dir = master_dir + 'run5/'

peptides_listfile = output_dir + 'peptides_list_Jihoon-run5-KRAS_muts.txt'
pmhc_template_pdbfile = master_dir + 'stuff_for_jihoon_2021-06-04/5yxn.pdb'
weights_filetag = 'ref2015_cart.wts' # This is the default

outfile_prefix = 'run5'

### more imports
import pandas as pd
import numpy as np
import sys
import pyrosetta

from pyrosetta.rosetta import core, protocols, numeric, basic, utility
from pyrosetta.rosetta.utility import vector1_bool as bools

# pyrosetta init
init_flags = '-ignore_unrecognized_res 1 -include_current'

#if args.ex1:
#    init_flags += f' -ex1'
#if args.ex2:
#    init_flags += f' -ex2'

pyrosetta.init(init_flags)

PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.ubuntu 2021.18+release.54b4909cd528ede1d749ea69e8046c244fc797f2 2021-05-04T21:04:43] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.ubuntu r282 2021.18+release.54b4909cd52 54b4909cd528ede1d749ea69e8046c244fc797f2 http://www.pyrosetta.org 2021-05-04T21:04:43
core.init: command: PyRosetta -ignore_unrecognized_res 1 -include_current -database /home/jwlee/.local/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-2076909332 seed_offset=0 real_seed=-2076909332
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-2076909332 RG_type=mt19937


In [2]:
################################################################################
# FUNCTIONS
################################################################################

def mutate_peptide_sequence( peptide_sequence, pose ):
    ''' This function mutates the current peptide sequence in pose
    to peptide_sequence
    '''
    old_sequence = pose.chain_sequence(2)
    print(f'mutate_peptide_sequence: from {old_sequence} to',
          peptide_sequence)

    pep_begin = pose.chain_begin(2)
    pep_end = pose.chain_end(2)

    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    if all(x in amino_acids for x in peptide_sequence): # all canonical AAs
        assert len(peptide_sequence) == pep_end-pep_begin+1

        tf = core.pack.task.TaskFactory()

        # freeze non-peptide:
        op = core.pack.task.operation.PreventRepacking()
        for i in range(1,pose.size()+1):
            if pose.chain(i) != 2:
                op.include_residue(i)
        tf.push_back(op)

        # force desired sequence at peptide
        for i, aa in zip(range(pose.chain_begin(2), pose.chain_end(2)+1),
                         peptide_sequence):
            op = core.pack.task.operation.RestrictAbsentCanonicalAAS()
            op.include_residue(i)
            op.keep_aas(aa)
            tf.push_back(op)

        packer = protocols.minimization_packing.PackRotamersMover()
        packer.task_factory(tf)

        packer.apply(pose)
    else:
        peptide_pose = core.pose.Pose()
        core.pose.make_pose_from_sequence(
            peptide_pose, peptide_sequence, "fa_standard")
        for i in range(len(old_sequence)):
            old_rsd = pose.residue(pep_begin+i)
            new_rsd = peptide_pose.residue(1+i)
            if old_rsd.name() != new_rsd.name():
                print('residue change:', old_rsd.name(), new_rsd.name())
                pose.replace_residue(pep_begin+i, new_rsd, True)


def fastrelax_peptide(
        scorefxn,
        nrepeats,
        peptide_positions,
        neighbor_positions,
        pose
):
    ''' This function optimizes the rosetta energy of the mhc-peptide complex
    It allows the amino acid side chains to change and all atoms in the peptide
    and nearby to move a small amount
    '''
    # movemap:
    mm = pyrosetta.rosetta.core.kinematics.MoveMap()
    mm.set_bb(False)
    mm.set_chi(False)
    mm.set_jump(False)

    for i in peptide_positions:
        mm.set_bb(i, True)
        mm.set_chi(i, True)

    for i in neighbor_positions:
        mm.set_chi(i, True)

    print('fastrelax_peptide:: adding chi flex at',
          len([x for x in neighbor_positions
               if x not in set(peptide_positions)]))

    fr = protocols.relax.FastRelax(scorefxn_in=scorefxn,
                                   standard_repeats=nrepeats)
    fr.cartesian(True)
    fr.set_movemap(mm)
    fr.set_movemap_disables_packing_of_fixed_chi_positions(True)

    # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"
    fr.min_type("lbfgs_armijo_nonmonotone")
    fr.apply(pose)


def unbind_peptide(pose):
    ''' This function slides the peptide away from the MHC so we can
    compute an unbound energy
    '''
    posl = range(1,pose.size()+1)

    chains = np.array([pose.chain(x) for x in posl])
    calphas = np.array([pose.residue(x).xyz("CA") for x in posl])

    mhc_cen = np.mean(calphas[chains==1], axis=0)
    pep_cen = np.mean(calphas[chains==2], axis=0)

    trans = pep_cen - mhc_cen
    trans = numeric.xyzVector_double_t(trans[0], trans[1], trans[2])

    j = pose.jump(1)
    j.translation_along_axis(pose.conformation().upstream_jump_stub(1), trans,
                             25.0)

    pose.set_jump(1, j)



def find_neighbors(core_positions, pose, heavyatom_distance_threshold = 6.0):
    ''' This function finds MHC residues that are nearby the peptide
    They will be allowed to move during the energy optimization
    '''
    # include all the 'core' positions as neighbors (of themselves, e.g.)
    nbr_positions = set(core_positions)

    posl = range(1, pose.size()+1)

    for i in posl:
        rsd1 = pose.residue(i)
        if rsd1.is_virtual_residue():
            continue
        for j in core_positions:
            rsd2 = pose.residue(j)
            if ( rsd1.nbr_atom_xyz().distance_squared( rsd2.nbr_atom_xyz() ) <=
                 ( rsd1.nbr_radius() + rsd2.nbr_radius() +
                   heavyatom_distance_threshold )**2):
                nbr_positions.add(i)
                break
    return nbr_positions


def read_pmhc_pose(filename, special=None):
    # 5yxn has the 5th chain as the 
    ''' This function reads a PDB file and deletes everything except the
    MHC peptide binding domain and the peptide

    Three possibilities for the input pdb:
    1) it has 3 chains: MHC, B2M, peptide
    2) it has 5 chains: MHC, B2M, peptide, TCRa, TCRb
    3) it has 2 chains: MHC, peptide

    '''
    pmhc_pose = pyrosetta.pose_from_pdb(filename)

    # delete second domain of MHC
    if pmhc_pose.chain_end(1) >= 180:
        pmhc_pose.delete_residue_range_slow(180, pmhc_pose.chain_end(1))
        core.pose.add_variant_type_to_pose_residue(
            pmhc_pose, core.chemical.UPPER_TERMINUS_VARIANT,
            pmhc_pose.chain_end(1));

    if special=='5yxn':
        # TCRa, TCRb, MHC, B2M, peptide (doesn't follow the above order)
        
        # Delete B2M
        pmhc_pose.delete_residue_range_slow(
            pmhc_pose.chain_begin(4), pmhc_pose.chain_end(4))
        # Delete TCR
        pmhc_pose.delete_residue_range_slow(
            pmhc_pose.chain_begin(1), pmhc_pose.chain_end(2))
    
    elif pmhc_pose.num_chains()==3:
        # MHC, B2M, peptide
        # delete B2M
        pmhc_pose.delete_residue_range_slow(
            pmhc_pose.chain_begin(2), pmhc_pose.chain_end(2))

    elif pmhc_pose.num_chains() == 5:
        # MHC. B2M, peptide, TCRa, TCRb
        # delete B2M
        pmhc_pose.delete_residue_range_slow(
            pmhc_pose.chain_begin(2), pmhc_pose.chain_end(2))

        # delete TCR
        pmhc_pose.delete_residue_range_slow(
            pmhc_pose.chain_begin(3), pmhc_pose.size())

    assert pmhc_pose.num_chains() == 2
    return pmhc_pose

Functions are defined, so now we use them below

In [8]:
pmhc_pose = read_pmhc_pose(pmhc_template_pdbfile, special='5yxn')

# find neighbor positions
peptide_positions = list(range(pmhc_pose.chain_begin(2),
                               pmhc_pose.chain_end(2)+1))
neighbor_positions = find_neighbors(peptide_positions, pmhc_pose)


template_peptide = pmhc_pose.chain_sequence(2)

with open(peptides_listfile, 'r') as data:
    peptides = [x[:-1] for x in data]

scorefxn = pyrosetta.create_score_function(weights_filetag)
peptide_scores = []

for peptide in peptides:
    #if len(peptide) != len(template_peptide)
    #: This is already checked in the mutate_peptide_sequence function
    #    print(f'ERROR: cant model peptide of mismatched length:',
    #          f'{peptide} on {template_peptide}')
    #    continue

    pose = pmhc_pose.clone()

    mutate_peptide_sequence(peptide, pose)

    fastrelax_peptide(scorefxn, 1, peptide_positions, neighbor_positions, pose)
    bound_score = scorefxn(pose)

    unbound_pose = pose.clone()

    unbind_peptide(unbound_pose)
    unbound_score_frozen = scorefxn(unbound_pose)

    fastrelax_peptide(scorefxn, 1, peptide_positions, neighbor_positions,
                      unbound_pose)
    unbound_score = scorefxn(unbound_pose)

    binding_energy = bound_score - unbound_score
    binding_energy_frozen = bound_score - unbound_score_frozen


    print('annotated_sequence:', pose.annotated_sequence())
    print('bound_score:', bound_score)
    print('unbound_score:', unbound_score)
    print('unbound_score_frozen:', unbound_score_frozen)
    print('binding_energy:', peptide, binding_energy)
    print('binding_energy_frozen:', peptide, binding_energy_frozen)
    print('bound_energies:', peptide,
          pose.energies().total_energies().weighted_string_of(
              scorefxn.weights()))
    print('unbound_energies:', peptide,
          unbound_pose.energies().total_energies().weighted_string_of(
              scorefxn.weights()))

    peptide_scores.append(dict(
        peptide=peptide,
        template=pmhc_template_pdbfile,
        bound_score=bound_score,
        unbound_score=unbound_score,
        unbound_score_frozen=unbound_score_frozen,
        binding_energy=binding_energy,
        binding_energy_frozen=binding_energy_frozen,
    ))


    if outfile_prefix is not None:
        # save the final pdb file
        peptide_for_filename = peptide.replace('[','_')\
                                      .replace(']','_')\
                                      .replace(':','_')
        outfile = f'{outfile_prefix}_{peptide_for_filename}.pdb'
        pose.dump_pdb(outfile)

        # write out the accumulate scores (we keep over-writing the old scores)
        outfile = f'{outfile_prefix}_scores.tsv'
        pd.DataFrame(peptide_scores).to_csv(outfile, sep='\t', index=False)

    sys.stdout.flush()



print('DONE')

core.import_pose.import_pose: File 'stuff_for_jihoon_2021-06-04/5yxn.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 192
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ASP:CtermProteinFull 433
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CB  on residue GLU 610
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 610
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 610
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 610
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 610
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 701
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 701
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE

mutate_peptide_sequence: from KLVALGINAV to KLVVVGAVGV
protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.pack_rotamers: built 13 rotamers at 10 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
fastrelax_peptide:: adding chi flex at 121
core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (285)
protocols.relax.FastRelax: CMD: repeat  234.695  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  234.695  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  -216.613  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1819 rotamers at 131 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -380.415  0  0  0.022
protocols.rela

protocols.relax.FastRelax: CMD: coord_cst_weight  -332.932  0.131451  0.131451  0.154
protocols.relax.FastRelax: CMD: scale:fa_rep  -255.141  0.131451  0.131451  0.30745
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1585 rotamers at 131 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -255.141  0.131451  0.131451  0.30745
protocols.relax.FastRelax: CMD: scale:fa_rep  -249.007  0.131451  0.131451  0.31955
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 101 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 164 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 203 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connect

core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 259 BRANCH 1
protocols.relax.FastRelax: CMD: min  -313.948  0.192146  0.192146  0.31955
protocols.relax.FastRelax: CMD: coord_cst_weight  -313.948  0.192146  0.192146  0.31955
protocols.relax.FastRelax: CMD: scale:fa_rep  -206.381  0.192146  0.192146  0.55
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1524 rotamers at 131 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -206.497  0.192146  0.192146  0.55
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 101 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 164 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: re

mutate_peptide_sequence: from KLVALGINAV to KLVVVGADGV
protocols.minimization_packing.PackRotamersMover: [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.pack_rotamers: built 22 rotamers at 10 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph
fastrelax_peptide:: adding chi flex at 121
core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (285)
protocols.relax.FastRelax: CMD: repeat  238.699  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  238.699  0  0  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  -211.596  0  0  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1826 rotamers at 131 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -374.052  0  0  0.022
protocols.rela

protocols.relax.FastRelax: CMD: coord_cst_weight  -332.131  0.10066  0.10066  0.154
protocols.relax.FastRelax: CMD: scale:fa_rep  -253.956  0.10066  0.10066  0.30745
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1596 rotamers at 131 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -253.956  0.10066  0.10066  0.30745
protocols.relax.FastRelax: CMD: scale:fa_rep  -247.792  0.10066  0.10066  0.31955
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 101 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 164 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 203 BRANCH 1
core.pose.util: [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angl

In [4]:
peptides_listfile

'peptides_list_Jihoon-run5-KRAS_muts.txt'

In [7]:
peptide

''